# Improved Hybrid CNN+MLP Training (V2)

Training improved version of Hybrid CNN+MLP model:
- Increased CNN capacity: 64→128→256 channels
- Increased MLP capacity: 512→256→128 neurons
- Residual connections in CNN branch
- Improved fusion layers: 256+128→256→128→64→2
- Better training: 80-100 epochs, warmup, cosine annealing, gradient clipping


In [1]:
import sys
from pathlib import Path
import torch
import torch.nn as nn
import json

PROJECT_ROOT = Path('/Volumes/SSanDisk/SpeechRec-German/without_context_windows')
sys.path.insert(0, str(PROJECT_ROOT))

from models.hybrid.hybrid_cnn_mlp_v2 import HybridCNNMLP_V2
from utils.training_utils import train_model, evaluate_model, WarmupCosineScheduler, LabelSmoothingCrossEntropy
from utils.data_loader import load_data, create_dataloaders

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print(f"Using MPS device")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

df, spectrograms_dict, feature_cols, feature_scaler, class_weights_dict = load_data(PROJECT_ROOT)
dataloaders = create_dataloaders(df, spectrograms_dict, feature_cols, feature_scaler, class_weights_dict, batch_size=64)

train_hybrid_loader = dataloaders['hybrid']['train']
val_hybrid_loader = dataloaders['hybrid']['val']
test_hybrid_loader = dataloaders['hybrid']['test']

OUTPUT_DIR = PROJECT_ROOT / 'artifacts' / 'b-p_dl_models' / 'improved_models'
class_weights = torch.tensor([class_weights_dict.get('0', class_weights_dict.get(0, 1.0)), 
                              class_weights_dict.get('1', class_weights_dict.get(1, 1.0))], dtype=torch.float32).to(device)


Using MPS device
Columns in df_phonemes: ['phoneme_id', 'utterance_id', 'phoneme', 'class', 'start_ms', 'end_ms', 'duration_ms', 'audio_path']
Columns in df_features: ['energy_rms', 'energy_rms_std', 'energy_zcr', 'energy_zcr_std', 'spectral_centroid', 'spectral_centroid_std', 'spectral_rolloff', 'spectral_rolloff_std', 'spectral_bandwidth', 'spectral_bandwidth_std', 'formant_f1', 'formant_f2', 'formant_f3', 'formant_f4', 'formant_f1_std', 'formant_f2_std', 'formant_f3_std', 'formant_f4_std', 'spectral_flatness', 'harmonic_noise_ratio', 'zcr_mean', 'energy_cv', 'phoneme_id', 'class', 'duration_ms', 'mfcc_mean_0', 'mfcc_mean_1', 'mfcc_mean_2', 'mfcc_mean_3', 'mfcc_mean_4', 'mfcc_mean_5', 'mfcc_mean_6', 'mfcc_mean_7', 'mfcc_mean_8', 'mfcc_mean_9', 'mfcc_mean_10', 'mfcc_mean_11', 'mfcc_mean_12', 'mfcc_std_0', 'mfcc_std_1', 'mfcc_std_2', 'mfcc_std_3', 'mfcc_std_4', 'mfcc_std_5', 'mfcc_std_6', 'mfcc_std_7', 'mfcc_std_8', 'mfcc_std_9', 'mfcc_std_10', 'mfcc_std_11', 'mfcc_std_12', 'delta_mfcc

## Model: Improved Hybrid CNN+MLP V2


In [2]:
model = HybridCNNMLP_V2(n_features=len(feature_cols), num_classes=2, dropout=0.2).to(device)

# Print model info
total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Model: {model.get_config()['model_type']}")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_params:,}")

# Loss function with label smoothing
criterion = LabelSmoothingCrossEntropy(smoothing=0.1, weight=class_weights)

# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)

# Learning rate scheduler with warmup and cosine annealing
num_epochs = 90
warmup_epochs = 5
scheduler = WarmupCosineScheduler(optimizer, warmup_epochs=warmup_epochs, total_epochs=num_epochs, min_lr=1e-6)

save_dir = OUTPUT_DIR / 'hybrid_cnn_mlp_v2'
save_dir.mkdir(parents=True, exist_ok=True)

print(f"\nTraining configuration:")
print(f"- Epochs: {num_epochs}")
print(f"- Warmup epochs: {warmup_epochs}")
print(f"- Initial LR: {optimizer.param_groups[0]['lr']}")
print(f"- Label smoothing: 0.1")
print(f"- Gradient clipping: 1.0")
print(f"- Early stopping patience: 15")


Model: HybridCNNMLP_V2
Total parameters: 1,513,922
Trainable parameters: 1,513,922

Training configuration:
- Epochs: 90
- Warmup epochs: 5
- Initial LR: 0.001
- Label smoothing: 0.1
- Gradient clipping: 1.0
- Early stopping patience: 15


In [3]:
history, best_epoch = train_model(
    model, train_hybrid_loader, val_hybrid_loader, criterion, optimizer, scheduler,
    device, num_epochs=num_epochs, save_dir=save_dir, model_name='hybrid_cnn_mlp_v2', 
    early_stopping_patience=15, max_grad_norm=1.0
)

checkpoint = torch.load(save_dir / 'best_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
test_metrics, _, _, _ = evaluate_model(model, test_hybrid_loader, criterion, device)

with open(save_dir / 'test_metrics.json', 'w') as f:
    json.dump(test_metrics, f, indent=2)

print(f"\n{'='*60}")
print(f"Final Test Results:")
print(f"{'='*60}")
print(f"Accuracy: {test_metrics['accuracy']:.4f}")
print(f"F1-score: {test_metrics['f1']:.4f}")
print(f"ROC-AUC: {test_metrics['roc_auc']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall: {test_metrics['recall']:.4f}")
print(f"Best epoch: {best_epoch}")



Epoch 1/90
--------------------------------------------------


Train Loss: 0.3891, Train Acc: 0.8998
Val Loss: 0.3460, Val Acc: 0.9282
Val F1: 0.9287, Val ROC-AUC: 0.9770
Learning Rate: 0.000200
✓ New best model saved! (F1: 0.9287)

Epoch 2/90
--------------------------------------------------


Train Loss: 0.3489, Train Acc: 0.9285
Val Loss: 0.3299, Val Acc: 0.9258
Val F1: 0.9271, Val ROC-AUC: 0.9831
Learning Rate: 0.000400

Epoch 3/90
--------------------------------------------------


Train Loss: 0.3410, Train Acc: 0.9327
Val Loss: 0.3338, Val Acc: 0.9235
Val F1: 0.9250, Val ROC-AUC: 0.9819
Learning Rate: 0.000600

Epoch 4/90
--------------------------------------------------


Train Loss: 0.3368, Train Acc: 0.9373
Val Loss: 0.3321, Val Acc: 0.9209
Val F1: 0.9226, Val ROC-AUC: 0.9833
Learning Rate: 0.000800

Epoch 5/90
--------------------------------------------------


...

Train Loss: 0.2389, Train Acc: 0.9971
Val Loss: 0.3360, Val Acc: 0.9520
Val F1: 0.9520, Val ROC-AUC: 0.9748
Learning Rate: 0.000013

Epoch 85/90
--------------------------------------------------


Train Loss: 0.2396, Train Acc: 0.9970
Val Loss: 0.3403, Val Acc: 0.9513
Val F1: 0.9511, Val ROC-AUC: 0.9731
Learning Rate: 0.000010

Epoch 86/90
--------------------------------------------------


Train Loss: 0.2384, Train Acc: 0.9972
Val Loss: 0.3390, Val Acc: 0.9524
Val F1: 0.9523, Val ROC-AUC: 0.9637
Learning Rate: 0.000006

Epoch 87/90
--------------------------------------------------


Train Loss: 0.2405, Train Acc: 0.9964
Val Loss: 0.3378, Val Acc: 0.9520
Val F1: 0.9519, Val ROC-AUC: 0.9692
Learning Rate: 0.000004

Early stopping at epoch 87
Best F1: 0.9533 at epoch 72



Final Test Results:
Accuracy: 0.9484
F1-score: 0.9481
ROC-AUC: 0.9838
Precision: 0.9481
Recall: 0.9484
Best epoch: 72
